In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.2'
# spark_version = 'spark-3.<spark version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:15 http://ppa

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-HW-2").getOrCreate()

# Load Amazon Data into Spark DataFrame

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Sports_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
sports_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Sports_v1_00.tsv.gz"), sep="\t", header=True)
sports_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   48945260|R1WBPB8MDCCN8F|B012P7UPSM|     409940130|Chicago Blackhawk...|          Sports|          5|            0|          0|   N|                N|   LOVE IT. 6 stars!|Bought this last ...| 2015-08-31|
|         US|    5782091|R32M0YEWV77XG8|B001GQ3VHG|     657746679|Copag Poker Size ...|          Sports|          5|    

# Filter by Votes

In [5]:
filtered_sports_df = sports_df.select(["star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
filtered_sports_df.show(10)

+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          5|            0|          0|   N|                N|
|          5|            1|          1|   N|                Y|
|          1|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            0|          1|   N|                N|
|          3|            0|          0|   N|                Y|
|          4|            2|          3|   N|                N|
|          5|            0|          0|   N|                Y|
|          5|            1|          1|   N|                Y|
|          5|            0|          0|   N|                Y|
+-----------+-------------+-----------+----+-----------------+
only showing top 10 rows



In [6]:
# Filter for greater than 20 total votes
total_votes_df = filtered_sports_df.filter(filtered_sports_df["total_votes"] >= 20)
total_votes_df.show(10)

+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          5|           32|         32|   N|                N|
|          5|           21|         21|   N|                Y|
|          2|           16|         20|   N|                Y|
|          5|          124|        128|   N|                Y|
|          1|           48|         65|   N|                Y|
|          4|           83|         89|   N|                Y|
|          5|           17|         22|   N|                N|
|          5|           26|         28|   N|                Y|
|          4|           22|         22|   N|                N|
|          5|           30|         31|   N|                Y|
+-----------+-------------+-----------+----+-----------------+
only showing top 10 rows



In [7]:
# Filter for greater than 50% helpful vote percentage
helpful_votes_df = total_votes_df.filter(total_votes_df["helpful_votes"]/total_votes_df["total_votes"] >= 0.5)
helpful_votes_df.show(10)

+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          5|           32|         32|   N|                N|
|          5|           21|         21|   N|                Y|
|          2|           16|         20|   N|                Y|
|          5|          124|        128|   N|                Y|
|          1|           48|         65|   N|                Y|
|          4|           83|         89|   N|                Y|
|          5|           17|         22|   N|                N|
|          5|           26|         28|   N|                Y|
|          4|           22|         22|   N|                N|
|          5|           30|         31|   N|                Y|
+-----------+-------------+-----------+----+-----------------+
only showing top 10 rows



# Describe Stats

In [8]:
# Paid reviews
paid_df = helpful_votes_df.filter(helpful_votes_df['vine']== 'Y')
paid_df.describe().show(10)

+-------+------------------+-----------------+-----------------+----+-----------------+
|summary|       star_rating|    helpful_votes|      total_votes|vine|verified_purchase|
+-------+------------------+-----------------+-----------------+----+-----------------+
|  count|               334|              334|              334| 334|              334|
|   mean| 4.098802395209581|65.01497005988024|72.36826347305389|null|             null|
| stddev|0.9737385324633377|84.58385900781398|94.57132147550898|null|             null|
|    min|                 1|              100|              100|   Y|                N|
|    max|                 5|               99|               99|   Y|                Y|
+-------+------------------+-----------------+-----------------+----+-----------------+



In [9]:
# Unpaid reviews
unpaid_df = helpful_votes_df.filter(helpful_votes_df['vine']== 'N')
unpaid_df.describe().show()

+-------+------------------+-----------------+------------------+-----+-----------------+
|summary|       star_rating|    helpful_votes|       total_votes| vine|verified_purchase|
+-------+------------------+-----------------+------------------+-----+-----------------+
|  count|             61614|            61614|             61614|61614|            61614|
|   mean|3.8771058525659754|50.94887525562372|56.097558996332005| null|             null|
| stddev|1.4764334185751982|103.9569927953232|108.97394149772389| null|             null|
|    min|                 1|               10|               100|    N|                N|
|    max|                 5|              998|               996|    N|                Y|
+-------+------------------+-----------------+------------------+-----+-----------------+



# Determine the percentage of five-star review among vine reviews

In [10]:
paid_five_star_number = paid_df[paid_df['star_rating']== 5].count()
paid_number = paid_df.count()
percentage_five_star_vine = round(float(paid_five_star_number) / float(paid_number),4)
print(f'Number of paid reviews {paid_number}')
print(f'Number of paid five star reviews {paid_five_star_number}')
print(f'Percantage of paid reviews that are five stars {percentage_five_star_vine * 100}%')

Number of paid reviews 334
Number of paid five star reviews 139
Percantage of paid reviews that are five stars 41.620000000000005%


# Determine the percentage of five-star review among non-Vine reviews

In [11]:
unpaid_five_star_number = unpaid_df[unpaid_df['star_rating']== 5].count()
unpaid_number = unpaid_df.count()
percentage_five_star_non_vine = round(float(unpaid_five_star_number) / float(unpaid_number),4)
print(f'Number of unpaid reviews {unpaid_number}')
print(f'Number of unpaid five star reviews {unpaid_five_star_number}')
print(f'Percantage of paid reviews that are five stars {percentage_five_star_non_vine * 100}%')

Number of unpaid reviews 61614
Number of unpaid five star reviews 32665
Percantage of paid reviews that are five stars 53.02%
